In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import matplotlib.pyplot as plt

In [12]:
data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
data.head()

In [13]:
data = np.array(data) #so that we can do maths with our data
data

In [14]:
m, n = data.shape #m = no. of rows (examples), n = no. of cols (pixels, =784 since 24*24)
np.random.shuffle(data)

# #for cross validation:
# data_dev = data[0:1000,:].T #take the 1st 1000 examples i.e. rows, and all columns; after that, transpose the matrix
# Y_dev = data_dev[0,:] #now the rows represent one pixel not one example, and first row is the labels
# X_dev = data_dev[1:n,:] #rest of the rows are all the pixels

# #for training
# data_train = data[1000:m,:].T #rest of the examples from 1000 to m will make up our training data
# Y_train = data_train[0,:]
# X_train = data_train[1:n,:]

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [15]:
print(data.shape)
print(data_dev.shape)
print(data_train.shape)

print(Y_dev.shape)
print(X_dev.shape)
print(Y_train.shape)
print(X_train.shape)
print(X_train[0].shape)
print(X_train[:, 0].shape)

In [16]:
def init_params(): #Initialise paramters
    W1 = np.random.rand(10, 784) - 0.5 #rand outputs between 0 and 1, we subtract 0.5 to make it between -0.5 to 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    #USING THE BELOW CODE CAUSES OUR NETWORK TO NOT UPDATE ANYTHING - FIND OUT WHY
#     W1 = np.random.randn(10, 784) - 0.5 #randn instead of rand
#     b1 = np.random.randn(10, 1) - 0.5
#     W2 = np.random.randn(10, 10) - 0.5
#     b2 = np.random.randn(10, 1) - 0.5
    
    return W1, b1, W2, b2
    
def ReLU(H):
    return np.maximum(H, 0) #element wise, goes through each element and returns the greater term of (element, 0)
    
def softmax(H):
    buffer = np.exp(H) / sum( np.exp(H) )
    return buffer
    # exp() applies to Z element wise
    # sum() applies to each column, hence preserves the amount of columns and collapses the amount of rows
    
def one_hot_encode(Y):
    one_hot_Y = np.zeros( (Y.size, Y.max() + 1 ) ) 
    #rows = no. of examples in Y
    #cols = no. of output classes = max label value + 1, since the labels start from 0
    one_hot_Y[np.arange(Y.size), Y] = 1
    #np.arange makes a list of numbers from 0 ... no. of examples, this specifies what row to access
    #Y, our labels, store which class is correct from 0-9. Hence, it gives the corresponding column to access for each row
    #after we have the col (class) for each row (example), we set it to 1 while the others remain 0
    one_hot_Y = one_hot_Y.T #we want each column to be an example instead of each row
    return one_hot_Y

def forward_prop(W1, b1, W2, b2, X):
    H1 = W1.dot(X) + b1 #W1 is 10*784, X is 784*1, b1 is 10*1, H1 is 10*1 (1st layer is 10*1 i.e. 10 nodes)
    Z1 = ReLU(H1) #Z1 is 10*1
    
    H2 = W2.dot(Z1) + b2 #W2 is 10*10, Z1 is 10*1, b2 is 10*1, H2 is 10*1 (2nd layer is 10*1 i.e. 10 nodes)
    Z2 = softmax(H2) #this stores our prediction, Z2 is 10*1
    
    return H1, Z1, H2, Z2
    
def deriv_ReLU(Z):
    return Z > 0 #true converts to 1, false converts to 0
    #for each element, if it is >0, slope of relu=1 and if <0, slope of relu = 0
    
def back_prop(H1, Z1, H2, Z2, W2, X, Y):
#     m = Y.size
    one_hot_Y = one_hot_encode(Y)
    
    dH2 = Z2 - one_hot_Y
    dW2 = 1 / m * dH2.dot(Z1.T)
    db2 = 1 / m * np.sum(dH2)
    dH1 = W2.T.dot(dH2) * deriv_ReLU(H1)
    
    dW1 = 1 / m * dH1.dot(X.T)
    db1 = 1 / m * np.sum(dH1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    W2 = W2 - alpha * dW2
    b1 = b1 - alpha * db1
    b2 = b2 - alpha * db2
    
    return W1, b1, W2, b2

In [17]:
def get_predictions(Z2):
    return np.argmax(Z2, 0)

def get_accuracy(predictions, Y):
    print( f"Predictions: {predictions}, Expected value: {Y}" )
    return np.sum ( predictions == Y ) / Y.size
    
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    
    for i in range(iterations):
        H1, Z1, H2, Z2 = forward_prop( W1, b1, W2, b2, X )
        dW1, db1, dW2, db2 = back_prop( H1, Z1, H2, Z2, W2, X, Y )
        
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            print("Iteration: ", i)
            predn = get_predictions(Z2)
            print("Accuracy: ", get_accuracy( predn, Y ))
            
    return W1, b1, W2, b2

In [18]:
import warnings

#suppress warnings
warnings.filterwarnings('ignore')

W1, b1, W2, b2 = gradient_descent( X_train, Y_train, 500, 0.1)

In [19]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [20]:
#Let's look at a couple of examples:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

In [21]:
# let's find the accuracy on the dev set:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

Still 84% accuracy, so our model generalized from the training data pretty well.